##Requirements

In [ ]:
#Libraries import
import os
import sys
import random
import math
import warnings
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd
import glob
import numpy as np
import pandas as pd
from skimage.io import imread
from matplotlib.cm import get_cmap
from skimage.segmentation import mark_boundaries
from skimage.util import montage
from skimage.morphology import binary_opening, disk, label
import gc; gc.enable() # memory is tight

In [ ]:
#Masked RCNN repository
!git clone https://github.com/maxw1489/Mask_RCNN.git
os.chdir('Mask_RCNN')

# Importing Mask RCNN
sys.path.append(os.path.join('Mask_RCNN'))  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

# Data Wrangling


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_dir = '/content/drive/MyDrive/Airbus ship detection/AirbusShipDetection'
main_dir = '/content/working'
train_dir = os.path.join(data_dir, 'train_v2')
test_dir = os.path.join(data_dir, 'test_v2')

In [ ]:
#Sample data/ Whole data
debug = False

In [ ]:
from PIL import Image

from sklearn.model_selection import train_test_split
exclude_list = ['6384c3e78.jpg','13703f040.jpg', '14715c06d.jpg',  '33e0ff2d5.jpg',
                '4d4e09f2a.jpg', '877691df8.jpg', '8b909bb20.jpg', 'a8d99130e.jpg',
                'ad55c3143.jpg', 'c8260c541.jpg', 'd6c7f17c7.jpg', 'dc3e7c901.jpg',
                'e44dffe88.jpg', 'ef87bad36.jpg', 'f083256d8.jpg'] #corrupted images

def filter_file_names(directory, exclude_list):
    file_names = []
    with os.scandir(directory) as entries:
        for entry in entries:
            if entry.name not in exclude_list and entry.is_file():
                file_names.append(entry.name)
    return file_names

train_names = filter_file_names(train_dir, exclude_list)
test_names = filter_file_names(test_dir, exclude_list)

print(len(train_names), len(test_names))


In [ ]:
# Convert the list to a DataFrame
df = pd.DataFrame({'File Name': train_names})

# Write the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Airbus ship detection/train_names.csv', index=False)


# Convert the list to a DataFrame
df = pd.DataFrame({'File Name': test_names})

# Write the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Airbus ship detection/test_names.csv', index=False)

In [ ]:

df = pd.read_csv('/content/drive/MyDrive/Airbus ship detection/train_names.csv')

train_names1 = df['File Name']


df = pd.read_csv('/content/drive/MyDrive/Airbus ship detection/test_names.csv')

test_names1 = df['File Name']



In [ ]:
#Code to filter available images in train_ship_segmentation file

# Path to the folder containing the images
image_folder = '/content/drive/MyDrive/Airbus ship detection/airbus-ship-detection/train_v2'

# Path to the CSV file
csv_file = '/content/drive/MyDrive/Airbus ship detection/airbus-ship-detection/train_ship_segmentations_v2.csv'

# Get the list of image names in the folder
image_names = [os.path.splitext(filename)[0] + '.jpg' for filename in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, filename))]

# Read the CSV file
data = pd.read_csv(csv_file)

# Filter the data based on image names
filtered_data = data[data['ImageId'].isin(image_names)]

# Save the filtered data to a new CSV file
filtered_data.to_csv('/content/drive/MyDrive/Airbus ship detection/airbus-ship-detection/train_ship_segmentations_v3.csv',index=False)

In [ ]:
# training dataset
rle = data_dir + '/train_ship_segmentations_v3.csv'
annotations = pd.read_csv(rle)


Gutcheck images

In [ ]:
im_names, image_annotations = train_names1, annotations


In [ ]:
ds = imread(os.path.join(train_dir, im_names[60])) # read  image from filepath
_ = plt.imshow(ds)

In [ ]:
# Original image size: 768 x 768
ORIG_SIZE = ds.shape[0]
#ORIG_SIZE = 768

#Model Building

In [ ]:
#add coments for confirguration
class ModelConfig(Config):
    # Give the configuration a recognizable name
    NAME = 'Initial'
    GPU_COUNT = 1
    IMAGES_PER_GPU = 9

    BACKBONE = 'resnet50'

    NUM_CLASSES = 2  # background and ship classes

    IMAGE_MIN_DIM = 384  #image shapes
    IMAGE_MAX_DIM = 384
    RPN_ANCHOR_SCALES = (8, 16, 32, 64)  #different window sizes in RPN achors
    TRAIN_ROIS_PER_IMAGE = 64  # Number of region of interest (RoI) proposals  -- Reduce the roi's
    MAX_GT_INSTANCES = 2   # maximum number of ground truth instances that can be present in each image.
    DETECTION_MAX_INSTANCES = 15  #maximum number of predicted instances that can be detected in each image.
    DETECTION_MIN_CONFIDENCE = 0.95  #minimum confidence threshold
    DETECTION_NMS_THRESHOLD = 0.0  #IoU (Intersection over Union) threshold for non-maximum suppression (NMS)
    RUN_EAGERLY = True    #Runs eagerly a bit fast, may tradeoff acc

    STEPS_PER_EPOCH = 15 if debug else 150    #number of training steps to be performed in each epoch.
    VALIDATION_STEPS = 10 if debug else 125   #Number of validation steps

    ## Initializing losses weights
    LOSS_WEIGHTS = {
        "rpn_class_loss": 30.0,
        "rpn_bbox_loss": 0.8,
        "mrcnn_class_loss": 6.0,
        "mrcnn_bbox_loss": 1.0,
        "mrcnn_mask_loss": 1.2
    }

config = ModelConfig()
config.display()

 **EncodedPixels** - a list of pixels for ship segmentation in a compressed format (in run-length encoding format).

 EncodedPixels сonsists of pairs of values that contain a start position and a run length.
 E.g. '1 3' implies starting at pixel 1 and running a total of 3 pixels (1,2,3).

 A prediction of of "no ship in image" have a blank value in the EncodedPixels column.




In [ ]:

def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction


In [ ]:
class LoadDataset(utils.Dataset):
    """Dataset class for training our dataset.
    """

    def __init__(self, im_names, image_annotations, orig_height, orig_width):
        super().__init__(self)

        #to add classes
        self.add_class('ship', 1, 'Ship')

        #to add images nae & annotation
        for i, fp in enumerate(im_names):
            annotations = image_annotations.query('ImageId=="' + fp + '"')['EncodedPixels']
            self.add_image('ship', image_id=i, path=os.path.join(train_dir, fp),
                           annotations=annotations, orig_height=orig_height, orig_width=orig_width)

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def load_image(self, image_id):
        info = self.image_info[image_id]
        fp = info['path']
        image = imread(fp)
        # If grayscale. Convert to RGB for consistency.
        if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1)
        return image

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        annotations = info['annotations']
#         print(image_id, annotations)
        count = len(annotations)
        if count == 0:
            mask = np.zeros((info['orig_height'], info['orig_width'], 1), dtype=np.uint8)
            class_ids = np.zeros((1,), dtype=np.int32)
        else:
            mask = np.zeros((info['orig_height'], info['orig_width'], count), dtype=np.uint8)
            class_ids = np.zeros((count,), dtype=np.int32)
            for i, a in enumerate(annotations):
                mask[:, :, i] = rle_decode(a)
                class_ids[i] = 1
        return mask.astype(np.bool), class_ids.astype(np.int32)

In [ ]:
# Train test split. Only considers 100 images if debug is True
from sklearn.model_selection import train_test_split
train_names1 = annotations[annotations.EncodedPixels.notnull()].ImageId.unique().tolist()  ## override with ships

test_size = config.VALIDATION_STEPS * config.IMAGES_PER_GPU
im_names_train, im_names_val = train_test_split(train_names1, test_size=test_size, random_state=42)

if debug:
    im_names_train = im_names_train[:100]
    im_names_val = im_names_val[:100]
    test_names1 = test_names1[:100]



Preparing train and validation data usingLoadDataset class.

In [ ]:
%%time
# prepare the training dataset
dataset_train = LoadDataset(im_names_train, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_train.prepare()

In [ ]:
%%time
# prepare the validation dataset
dataset_val = LoadDataset(im_names_val, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_val.prepare()

Training

In [ ]:
model = modellib.MaskRCNN(mode='training', config=config, model_dir=main_dir)



In [ ]:
LEARNING_RATE = 0.003

# Train Mask-RCNN Model
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%time
## Only training classification, bounding box regression, and mask prediction head (ROI align)
model.train(dataset_train, dataset_val,
            learning_rate=LEARNING_RATE*2,
            epochs=2,
            layers='heads')

history = model.keras_model.history.history

#losses
#loss : Overall loss
#rpn_class_loss: loss associated with the classification of anchor boxes in Region Proposal Network (RPN) part
#rpn_bbox_loss: loss associated with refining the coordinates of the proposed bounding box regions by the RPN.
#mrcnn_class_loss: loss associated with the classification of objects by the Mask R-CNN network
#mrcnn_bbox_loss: loss associated with refining the coordinates of the bounding boxes predicted by the Mask R-CNN network.
#mrcnn_mask_loss: loss associated with the segmentation masks predicted by the Mask R-CNN network.

In [ ]:
#Training all layers. This includes both the backbone network and the heads
#%%time
 # LEARNING_RATE
model.train(dataset_train, dataset_val,
            learning_rate=LEARNING_RATE,
            epochs=4 if debug else 40,
            layers='all')

new_history = model.keras_model.history.history
for k in new_history: history[k] = history[k] + new_history[k]

In [ ]:
import pickle
save_path = '/content/drive/MyDrive/Airbus ship detection/model_history.pkl'

with open(save_path, 'wb') as file:
    pickle.dump(history, file)

In [ ]:
#model 1's history
epochs = range(1, len(new_history['loss'])+1)
pd.DataFrame(new_history, index=epochs)


In [ ]:
#Train vs validation plot
plt.figure(figsize=(21,11))

plt.subplot(231)
plt.plot(epochs, history["loss"], label="Train loss")
plt.plot(epochs, history["val_loss"], label="Valid loss")
plt.legend()
plt.subplot(232)
plt.plot(epochs, history["rpn_class_loss"], label="Train RPN class loss")
plt.plot(epochs, history["val_rpn_class_loss"], label="Valid RPN class loss")
plt.legend()
plt.subplot(233)
plt.plot(epochs, history["rpn_bbox_loss"], label="Train RPN box loss")
plt.plot(epochs, history["val_rpn_bbox_loss"], label="Valid RPN box loss")
plt.legend()
plt.subplot(234)
plt.plot(epochs, history["mrcnn_class_loss"], label="Train MRCNN class loss")
plt.plot(epochs, history["val_mrcnn_class_loss"], label="Valid MRCNN class loss")
plt.legend()
plt.subplot(235)
plt.plot(epochs, history["mrcnn_bbox_loss"], label="Train MRCNN box loss")
plt.plot(epochs, history["val_mrcnn_bbox_loss"], label="Valid MRCNN box loss")
plt.legend()
plt.subplot(236)
plt.plot(epochs, history["mrcnn_mask_loss"], label="Train Mask loss")
plt.plot(epochs, history["val_mrcnn_mask_loss"], label="Valid Mask loss")
plt.legend()

plt.show()

In [ ]:
best_epoch = np.argmin(history["val_loss"])
# From the plot. Set best epoch path here epoch 33
model_path = "/content/mask_rcnn_initial_0033.h5"


#Bounding box and confidence score

In [ ]:
class InferenceConfig(ModelConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode='inference',
                          config=inference_config,
                          model_dir=main_dir)

# Load trained weights (fill in path to trained weights here)
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# set color for class
def get_colors_for_class_ids(class_ids):
    colors = []
    for class_id in class_ids:
        if class_id == 1:
            colors.append((.941, .204, .204))
    return colors

In [ ]:
# Show few example of ground truth vs. predictions on the validation dataset
dataset = dataset_val
fig = plt.figure(figsize=(10, 40))

for i in range(4,6,1):

    image_id = random.choice(dataset.image_ids)

    original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,      #load original images along with ground truth
                               image_id, use_mini_mask=False)

# print(original_image.shape)
    plt.subplot(8, 2, 2*i + 1)
    visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id,
                                dataset.class_names,
                                colors=get_colors_for_class_ids(gt_class_id), ax=fig.axes[-1])

    plt.subplot(8, 2, 2*i + 2)
    results = model.detect([original_image]) #, verbose=1)
    r = results[0]
    visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'],
                                dataset.class_names, r['scores'],
                                colors=get_colors_for_class_ids(r['class_ids']), ax=fig.axes[-1])

Accuracy metrics

Map, IOU and F-2 score


In [ ]:

import numpy as np
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")
# Define a function to calculate AP, mAP, IoU, and  F2 score
def calculate_ap_map_iou_f2(model, dataset, inference_config):
    APs = []
    IoUs = []
    F2s = []
    gt_bbox_list = []
    gt_class_id_list = []
    gt_mask_list = []
    pred_bbox_list = []
    pred_class_id_list = []
    pred_score_list = []
    

    # Loop through the dataset
    for image_id in dataset.image_ids:
        original_image, image_meta, gt_class_id, gt_bbox, gt_mask = \
            modellib.load_image_gt(dataset, inference_config, image_id, use_mini_mask=False)

        # Append ground truth annotations to lists
        gt_bbox_list.append(gt_bbox)
        gt_class_id_list.append(gt_class_id)
        gt_mask_list.append(gt_mask)

        # Run inference on the image
        results = model.detect([original_image])
        r = results[0]

        # Append predicted results to lists
        pred_bbox = r['rois']
        pred_class_ids = r['class_ids']
        pred_scores = r['scores']
        pred_masks = r['masks']
        pred_bbox_list.append(pred_bbox)
        pred_class_id_list.append(pred_class_ids)
        pred_score_list.append(pred_scores)

        # Compute AP and IoU for the image
        AP, _, _, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask, pred_bbox, pred_class_ids, pred_scores, pred_masks, iou_threshold=0.5)  # Use IoU threshold of 0.5 for competition evaluation
        APs.append(AP)

        # Calculate IoU
        valid_overlaps = overlaps[~np.isnan(overlaps)]  # Exclude NaN IoU values
        if len(valid_overlaps) > 0:
            IoU = np.mean(valid_overlaps)
            IoUs.append(IoU)

        # Calculate  F2 score
        TP = len(overlaps) - np.isnan(overlaps).sum()  # True Positives (excluding NaN IoU values)
        FN = len(gt_bbox) - TP  # False Negatives
        FP = len(pred_bbox) - TP  # False Positives
        beta = 2  # F2 score weight for recall (increase weight of recall)
        F2 = ((1 + beta**2) * TP) / ((1 + beta**2) * TP + beta**2 * FN + FP)  # Modified Kaggle F2 score formula
        #F2s.append(F2)
        if F2 >= 1:  # Only consider F1 scores >= 1
            F2s.append(F2)

    # Calculate mean Average Precision (mAP), mean IoU (excluding NaN values), and mean  F2 score
    mAP = np.mean(APs)
    mean_IoU = np.nanmean(IoUs) if len(IoUs) > 0 else np.nan
    mean_F2 = np.mean(F2s)

    return APs, mAP, IoUs, mean_IoU, F2s, mean_F2

# Call the function to calculate AP, mAP, IoU, and  F2 score
APs, mAP, IoUs, mean_IoU, F2s, mean_F2 = calculate_ap_map_iou_f2(model, dataset_val, inference_config)

# Print the results
print("Average Precision (AP) per class:")
# for i, AP in enumerate(APs):
#     print(f"Class {i}: AP = {AP}")

print("Mean Average Precision (mAP):", mAP)

# print("Intersection over Union (IoU) per image:")
# for i, IoU in enumerate(IoUs):
#     print(f"Image {i}: IoU = {IoU}")

print("Mean IoU :", mean_IoU)

# print(" F2 score per image:")
# for i, F2 in enumerate(F2s):
#     print(f"Image {i}: F2 score = {F2}")

print("Mean F2 score:", mean_F2)
